In [8]:
from huggingface_hub import notebook_login
notebook_login()

In [1]:
from torch import cuda, bfloat16
import transformers
import os
from torch import cuda, bfloat16
import transformers
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)

model_id = 'meta-llama/Llama-2-7b-chat-hf'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# Need auth token for these
hf_auth = os.environ.get('hf_token')
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    # quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=hf_auth
)
model.eval()
print(f"Model loaded on {device}")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model loaded on cuda:0


In [2]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "right"

In [9]:
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "Given a n3.pcap file containing network traffic data, create a Python script to process the PCAP. The script should extract the source IP, destination IP, source port, destination port, and the raw payload data for each packet in the PCAP. Store this data in a pandas DataFrame and display it."
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1012)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Given a n3.pcap file containing network traffic data, create a Python script to process the PCAP. The script should extract the source IP, destination IP, source port, destination port, and the raw payload data for each packet in the PCAP. Store this data in a pandas DataFrame and display it. [/INST]  To process a PCAP file and extract the source IP, destination IP, source port, destination port, and raw payload data for each packet, you can use the `pcapy` library in Python. Here's an example of how you can do this:
```
import pcapy
import pandas as pd

# Open the PCAP file
with open("n3.pcap", "rb") as f:
    # Create a pcapy.Pcap object
    pcap = pcapy.Pcap(f)

    # Extract the packet data
    packets = []
    for packet in pcap.packets():
        # Extract the source IP, destination IP, source port, destination port, and raw payload data
        src_ip = packet.get_field("src_ip")
        dst_ip = packet.get_field("dst_ip")
        src_port = packet.get_field("src_port"

In [ ]:
import json
import re

input_file_path = "/home/hb/LLM-research/evaluation/5G_SPEC/5G_multi_choice.json" 

with open(input_file_path, "r") as input_file:
    questions_data = json.load(input_file)

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)

generated_answers = []
ground_truth = [q_dict["answer"] for q_dict in questions_data]
n_question = 0

for q_dict in questions_data:
    question = q_dict["question"]
    options = q_dict["choices"]
    
    # Construct prompt with question and options
    prompt = f"Choose the correct answer: {question}\n" + "\n".join(options)
    # print(prompt)
    
    # Generate answer using the language model
    generated_answer = pipe(prompt)[0]['generated_text']
    print(f"Generated Answer: {generated_answer}")
    
    match = re.search(r'(Answer:|Correct answer:)\s*((?:[A-D]\)\s*)?[\w\s()\-/]+)', generated_answer)
    if match:
        parsed_answer = match.group(2).strip()
        
        # Exclude the "Explanation" part from the parsed answer if present
        explanation_start = parsed_answer.find("\nExplanation")
        if explanation_start != -1:
            parsed_answer = parsed_answer[:explanation_start].strip()
        
        print(f"Parsed_answer: {parsed_answer}")
        generated_answers.append(parsed_answer)
    else:
        print("No answer detected.")
        generated_answers.append("N/A")
    
    n_question += 1
    print(f"----------------{n_question}-----------------")
print(generated_answers)


total_questions = len(questions_data)
correct_answers = 0

for ground_answer, generated_answer in zip(ground_truth, generated_answers):
    if generated_answer.lower() == ground_answer.lower():
        print("Correct")
        correct_answers += 1
    else:
        print(f"GPT answer: {generated_answer}. Correct answer: {ground_answer}")
        print("Incorrect")

accuracy = (correct_answers / total_questions) * 100

print(f"Accuracy: {accuracy:.2f}%")
print(f"Correct answers: {correct_answers}")
print(f"Incorrect answers: {total_questions - correct_answers}")

In [ ]:
import json

input_file_path = "/home/hb/LLM-research/evaluation/5G_SPEC/5G_SPEC_QA.json" 

with open(input_file_path, "r") as input_file:
    questions_data = json.load(input_file)

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=512)

generated_answers_qa = []

n_question = 0
for q_dict in questions_data:
    n_question += 1
    question = q_dict["question"]
    # Generate answer using the language model
    prompt = f"[INST]{question}[/INST]"
    generated_answer = pipe(prompt)[0]['generated_text']
    print(f"Output: {generated_answer}")
    generated_answers_qa.append(generated_answer)
    print(f"---------------------{n_question}-----------------------------")

`Network config QA`: Pass